In [1]:
import cv2
import numpy as np

# Load the video
video_path = "path_to_your_video.mp4"
cap = cv2.VideoCapture(video_path)

# Define a function to split the frame into quadrants
def split_into_quadrants(frame):
    height, width, _ = frame.shape
    half_height, half_width = height // 2, width // 2
    quadrants = {
        1: frame[:half_height, :half_width],
        2: frame[:half_height, half_width:],
        3: frame[half_height:, :half_width],
        4: frame[half_height:, half_width:]
    }
    return quadrants

# Define a function to detect balls by color
def detect_balls(frame, color_ranges):
    detected_balls = []
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    for color_name, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, lower, upper)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) > 100:  # Minimum area to filter noise
                x, y, w, h = cv2.boundingRect(contour)
                detected_balls.append((color_name, x + w // 2, y + h // 2))  # Ball center
    return detected_balls

# Define color ranges for different colored balls (HSV format)
color_ranges = {
    "red": (np.array([0, 120, 70]), np.array([10, 255, 255])),
    "green": (np.array([36, 100, 100]), np.array([70, 255, 255])),
    "blue": (np.array([94, 80, 2]), np.array([126, 255, 255]))
}

# Main loop to process the video
events = []
prev_positions = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    quadrants = split_into_quadrants(frame)
    for quadrant_num, quadrant_frame in quadrants.items():
        detected_balls = detect_balls(quadrant_frame, color_ranges)
        for color_name, x, y in detected_balls:
            global_x = x + (0 if quadrant_num in [1, 3] else frame.shape[1] // 2)
            global_y = y + (0 if quadrant_num in [1, 2] else frame.shape[0] // 2)
            current_pos = (global_x, global_y)

            if color_name in prev_positions:
                prev_pos = prev_positions[color_name]
                prev_quadrant = (prev_pos[1] < frame.shape[0] // 2) * 1 + (prev_pos[0] >= frame.shape[1] // 2) * 2 + (prev_pos[1] >= frame.shape[0] // 2) * 3
                if prev_quadrant != quadrant_num:
                    events.append((color_name, prev_quadrant, quadrant_num))
            prev_positions[color_name] = current_pos

cap.release()

# Display events
for event in events:
    print(f"Ball {event[0]} moved from quadrant {event[1]} to quadrant {event[2]}")